In [ ]:
#test code for input
import sys

#Algorithm for frequent pattern mining
from itertools import chain,combinations

def joinset(itemset, k):
    return(set([i.union(j) for i in itemset for j in itemset if len(i.union(j)) == k]))

def generating_subset(itemset,k):
    p = set()
    for item in combinations(itemset,k-1):
        plist = []
        for mo in item:
            plist.append(mo)
        p.add(frozenset(plist))
    return(p)

def freq_pattern(transaction_list, itemset, min_sup=0):
    l = [
        (item, int(sum(1 for row in transaction_list if item.issubset(row)))) 
        for item in itemset
    ]
    return dict([(item, support) for item, support in l if support >= min_sup])

def print_result(fset):
    #print('--Frequent Itemset--')
    #for item, support in sorted(fset.items()):
    #for item, support in (fset.items()):
        #print('{} : {}'.format(tuple(item),support)
         #print('{} [{}]'.format(support,' '.join(list(item))))
    for item,support in sorted(fset.items(), key= lambda item_support: (-item_support[1],(list(item_support[0])))):
        print('{} [{}]'.format(support,' '.join(map(str,list(item)))))
        #temp_list = list(item)
        #temp_list.sort()
        #print(temp_list)
        #print(support,' ',' '.join(map(str,list(item))))

def sort_dic(fset):
    newdic = dict()
    for item,support in (fset.items()):
        temp_list = list(item)
        temp_list.sort()
        newdic.update(dict([(tuple(temp_list),support)]))
    return(newdic)

def max_pattern(mset,cpset,min_sup):
    compset = set()
    #n = max(len(item) for item in tset)
    for item,support in (mset.items()):
        n = len(item)
        if (len(item)>1):
            for index in range(1,n+1):
                compset.update(generating_subset(item,index))
                compset.discard(frozenset())

    for item in compset:
        if (item in mset):
            del mset[item]
            if(not(cpset[item]>min_sup)):
                del cpset[item]
    

transaction_list = []
count_line =0
min_sup = 0

for line in sys.stdin:
    if(line):
        if(count_line==0):
            min_sup= int((line.split())[0])
            count_line = count_line+1
            continue
        new_list = [elem for elem in line.split()]
        transaction_list.append(new_list)

single_candidateset = set()
#min_sup = 2
#transaction_list= [['B','A','C','D'],['B','A','P','Q']]
#transaction_list= [['B','A','C','E','D'],['A','C'],['C','B','D']]
#transaction_list= [['data','mining'],['frequent','pattern','mining'],['mining','frequent','patterns'],['closed','and','maximal','pattern','mining']]
#print(transaction_list)
for row in transaction_list:
    for item in row:
        if item:
            single_candidateset.add(frozenset([item]))
#print (single_candidateset)

fset = dict()
mset = dict()
cpset = dict()
cset = single_candidateset
middle = set()
tracker = 1

while True:
    if(tracker>1):
        #join frequent item
        cset = joinset(lset,tracker)
    if (tracker>2):
        for item in cset:
            #prune infrequentsubsets
            check = generating_subset(item,tracker)
            if(not(check.issubset(lset))):
                middle.add(item)
        cset = cset-middle
    lset = freq_pattern(transaction_list,cset,min_sup)
    if not lset:
        break
    fset.update(lset)
    mset.update(lset)
    cpset.update(lset)
    tracker = tracker+1
    
max_pattern(mset,cpset,min_sup)
mset = sort_dic(mset)
fset = sort_dic(fset)
cpset = sort_dic(cpset)
#fset = sorted(fset.items(), key= lambda item_support: (-item_support[1],(list(item_support[0]))))
#print(fset)
print_result(fset)
print()
print_result(cpset)
print()
print_result(mset)